In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mouse_movement_metrics_CHI_2019.xlsx to mouse_movement_metrics_CHI_2019.xlsx


In [ ]:
import io
import pandas as pd
import numpy as np
df = pd.read_excel(io.BytesIO(uploaded['mouse_movement_metrics_CHI_2019.xlsx']))
df

,Unnamed: 0,session_id,appliance_id,appliance_name,rating,pause_before_click,back_n_forth,unclicked_hovers,unclicked_hovers_in_answer,back_n_forth_in_answer,loops,loops_in_answer,pauses,pauses_in_answer,pause_time
0,0,187,40,Washing machine,7,0.567,0,3,0,0,1,0,3,1,6.439
1,1,187,14,Electric blanket,8,7.883,0,0,0,0,0,0,1,1,7.883
2,2,187,8,Coffee maker,3,0.836,0,1,0,0,0,0,2,1,1.262
3,3,187,13,DVD player,3,9.903,0,0,0,0,0,0,1,1,9.903
4,4,187,32,Router,1,1.516,0,0,0,0,0,0,1,1,1.516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10327,10327,616,3,Blender,8,0.344,0,0,0,0,0,0,1,1,0.344
10328,10328,616,13,DVD player,5,1.124,0,0,0,0,0,0,1,1,1.124
10329,10329,616,32,Router,5,0.980,0,0,0,0,0,0,1,1,0.980
10330,10330,616,21,Lightbulb - CFL,6,0.892,0,0,0,0,0,0,1,1,0.892


In [ ]:
!pip install jenkspy
import jenkspy



     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for jenkspy: filename=jenkspy-0.2.0-cp37-cp37m-linux_x86_64.whl size=70325 sha256=19a6ac265db49be49bcaf1a3429fc44db0feda8c0cec2e937b1b7e25149146c4
  Stored in directory: /root/.cache/pip/wheels/e5/ab/c7/449bc908636d689818fca058a9b530f61220b634a09597a69d
Successfully built jenkspy


In [ ]:

headers = ['back_n_forth', 'unclicked_hovers',
       'unclicked_hovers_in_answer', 'back_n_forth_in_answer', 'loops',
       'loops_in_answer', 'pauses', 'pauses_in_answer', 'pause_time', 'pause_before_click']


In [ ]:
def f(row):
  a = row[i]
  if a >= breaks[0] and a < breaks[1]:
    val = 1
  elif a >= breaks[1] and a < breaks[2]:
    val = 2
  elif  a >= breaks[2] and a < breaks[3]:
    val = 3 
  elif a >= breaks[3] and a < breaks[4]:
    val = 4
  else:
    val = 5
  return val

In [ ]:
cluster_df = df

In [ ]:
#df[headers[0]+'_cluster'] = df['pauses']
#df[headers[0]+'_cluster'] = df[headers[0]+'_cluster'] +1
for i in headers:
  breaks = jenkspy.jenks_breaks(df[i], nb_class=5)
  cluster_df[i] = df.apply(f, axis=1)

In [ ]:

cluster_df



,Unnamed: 0,session_id,appliance_id,appliance_name,rating,pause_before_click,back_n_forth,unclicked_hovers,unclicked_hovers_in_answer,back_n_forth_in_answer,loops,loops_in_answer,pauses,pauses_in_answer,pause_time
0,0,187,40,Washing machine,7,1,2,4,2,2,3,2,3,3,3
1,1,187,14,Electric blanket,8,4,2,2,2,2,2,2,2,3,3
2,2,187,8,Coffee maker,3,1,2,3,2,2,2,2,3,3,1
3,3,187,13,DVD player,3,5,2,2,2,2,2,2,2,3,4
4,4,187,32,Router,1,2,2,2,2,2,2,2,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10327,10327,616,3,Blender,8,1,2,2,2,2,2,2,2,3,1
10328,10328,616,13,DVD player,5,2,2,2,2,2,2,2,2,3,1
10329,10329,616,32,Router,5,2,2,2,2,2,2,2,2,3,1
10330,10330,616,21,Lightbulb - CFL,6,1,2,2,2,2,2,2,2,3,1


In [ ]:
device = cluster_df.groupby('appliance_id').first()
del device['session_id']
del device['Unnamed: 0']
del device['rating']

for i in range(41):
  for j in headers:
    x = len(cluster_df[(cluster_df[j]>3) & (cluster_df["appliance_id"] == i+1)])
    device.loc[i+1, j] = x


In [ ]:

device['sum'] = device.sum(axis = 1, skipna = True)
output = device.sort_values(by = ['sum'],ascending=False).head(10)
output

,appliance_name,pause_before_click,back_n_forth,unclicked_hovers,unclicked_hovers_in_answer,back_n_forth_in_answer,loops,loops_in_answer,pauses,pauses_in_answer,pause_time,sum
appliance_id,,,,,,,,,,,,
33,Satelite dish,5,30,49,6,9,8,1,34,26,7,175
14,Electric blanket,5,25,50,10,6,4,0,36,30,3,169
28,Popcorn machine,1,29,49,8,8,5,2,33,28,5,168
21,Lightbulb - CFL,3,28,54,8,7,6,0,32,28,2,168
10,Desk computer,1,30,45,8,7,8,0,29,34,3,165
4,Cable TV converter,3,22,47,6,6,9,0,31,26,5,155
35,Stove,4,25,45,7,11,3,0,29,26,3,153
22,Laptop,1,30,43,9,12,4,1,25,25,1,151
38,TV - Flatscreen,1,26,38,5,10,5,0,31,31,1,148


In [ ]:
#device['appliance_no'] = device['appliance_id']

In [ ]:
df_JSI = pd.DataFrame()
for i in headers:
  top_app = device.sort_values(by = i, ascending=False).head(5)
  col = top_app['appliance_name'].values
  df_JSI[i] = col
df_JSI

,back_n_forth,unclicked_hovers,unclicked_hovers_in_answer,back_n_forth_in_answer,loops,loops_in_answer,pauses,pauses_in_answer,pause_time,pause_before_click
0,Desk computer,Lightbulb - CFL,Electric blanket,Laptop,Cable TV converter,Popcorn machine,Electric blanket,Desk computer,Satelite dish,Dishwasher
1,Laptop,Electric blanket,Laptop,Blender,Heater,Refrigerator - Large,Satelite dish,Freezer,DVD player,Water heater
2,Satelite dish,Satelite dish,Lightbulb - CFL,Stove,Vaccum cleaner,Satelite dish,Popcorn machine,TV - Flatscreen,Cable TV converter,Electric blanket
3,Popcorn machine,Curling iron,Washing machine,TV - Flatscreen,Satelite dish,Aquarium,Heater,Washing machine,Popcorn machine,Satelite dish
4,Lightbulb - CFL,Popcorn machine,Desk computer,DVD player,Desk computer,Lightbulb - ICL,Lightbulb - CFL,Electric blanket,Mixer - Table,Mixer - Table


In [ ]:
df_JSI.rename(columns={'pause_before_click': 'p_b_c', 'back_n_forth': 'b_n_f', 'unclicked_hovers': 'u_h',
       'unclicked_hovers_in_answer': 'u_h_a', 'back_n_forth_in_answer': 'b_n_f_a', 
       'loops_in_answer': 'l_a', 'pauses_in_answer': 'p_a'})

,b_n_f,u_h,u_h_a,b_n_f_a,loops,l_a,pauses,p_a,pause_time,p_b_c
0,Desk computer,Lightbulb - CFL,Electric blanket,Laptop,Cable TV converter,Popcorn machine,Electric blanket,Desk computer,Satelite dish,Dishwasher
1,Laptop,Electric blanket,Laptop,Blender,Heater,Refrigerator - Large,Satelite dish,Freezer,DVD player,Water heater
2,Satelite dish,Satelite dish,Lightbulb - CFL,Stove,Vaccum cleaner,Satelite dish,Popcorn machine,TV - Flatscreen,Cable TV converter,Electric blanket
3,Popcorn machine,Curling iron,Washing machine,TV - Flatscreen,Satelite dish,Aquarium,Heater,Washing machine,Popcorn machine,Satelite dish
4,Lightbulb - CFL,Popcorn machine,Desk computer,DVD player,Desk computer,Lightbulb - ICL,Lightbulb - CFL,Electric blanket,Mixer - Table,Mixer - Table


In [ ]:
JSI_result = pd.DataFrame()
JSI_result['Matrics'] = headers


In [ ]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
k=0
for i in headers:
  for j in headers:
    x = jaccard_similarity(df_JSI[i], df_JSI[j])
    x = round(x,2)
    JSI_result.loc[k, j] = x  
  k=k+1

      

In [ ]:
JSI_result

,Matrics,back_n_forth,unclicked_hovers,unclicked_hovers_in_answer,back_n_forth_in_answer,loops,loops_in_answer,pauses,pauses_in_answer,pause_time,pause_before_click
0,back_n_forth,1.00,0.43,0.43,0.11,0.25,0.25,0.43,0.11,0.25,0.11
1,unclicked_hovers,0.43,1.00,0.25,0.00,0.11,0.25,0.67,0.11,0.25,0.25
2,unclicked_hovers_in_answer,0.43,0.25,1.00,0.11,0.11,0.00,0.25,0.43,0.00,0.11
3,back_n_forth_in_answer,0.11,0.00,0.11,1.00,0.00,0.00,0.00,0.11,0.11,0.00
4,loops,0.25,0.11,0.11,0.00,1.00,0.11,0.25,0.11,0.25,0.11
5,loops_in_answer,0.25,0.25,0.00,0.00,0.11,1.00,0.25,0.00,0.25,0.11
6,pauses,0.43,0.67,0.25,0.00,0.25,0.25,1.00,0.11,0.25,0.25
7,pauses_in_answer,0.11,0.11,0.43,0.11,0.11,0.00,0.11,1.00,0.00,0.11
8,pause_time,0.25,0.25,0.00,0.11,0.25,0.25,0.25,0.00,1.00,0.25
9,pause_before_click,0.11,0.25,0.11,0.00,0.11,0.11,0.25,0.11,0.25,1.00


In [ ]:
k=0
max=0
min=1000
cnt=0
dt={}
dct={}
for i in headers:
  for j in headers:
    x = JSI_result.loc[k, j]
    if x!=1:
      dct[x] = dct.get(x, 0) + 1
      if x>max:
        max=x
      if x<min:
        min=x
      if x>.5:
        cnt=cnt+1
        a="("+i+", "+j+")"   
        dt[a]=x
  k=k+1

print("max: ",max)
print("min : ",min)
print("JSI range: " +str(min)+"<=JSI<=" + str(max))
print("no of JSI value more than .5: ",cnt)
print("JSI value more than .5: ", dt)
print("No of times each value is: ",dct)

max:  0.67
min :  0.0
JSI range: 0.0<=JSI<=0.67
no of JSI value more than .5:  2
JSI value more than .5:  {'(unclicked_hovers, pauses)': 0.67, '(pauses, unclicked_hovers)': 0.67}
No of times each value is:  {0.43: 8, 0.11: 32, 0.25: 30, 0.0: 18, 0.67: 2}
